In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch; import torch.nn as nn; import torch.nn.functional as F

# 12.1 Attention

At a very high level, the attention mechanism learns mappings from sequences of input tokens that map tokens into a semantic embedding space. This embedding space fundamentally encodes *relationships* between the input tokens based on their *values* and *positions* in the input sequences. The representation of a token in the embedding space (or latent space) is, therefore, dependent upon the other tokens in the input, and their positions.

## 12.1.1 Transformer Processing

The input data is a set of vectors $\{x_n\}, \ x_i \in \reals^D$. Each input vector is referred to as a *token*. Elements $x_{ni}$ of each token are called *features*. So, an input data matrix $X$ has dimensions $N\times D$ with $N$ tokens and $D$ features.\
Then, the transformer maps this input to an output matrix of the *same dimensionality*:
$$\tilde X = \text{TransformerLayer}[X]$$

In general, this transformer layer is comprised of two stages:
1. The attention mechanism - which mixes together features across different token vectors - column-wise function
2. Some token projection - row-wise function that transforms the features of each token vector

## 12.1.2 Attention Coefficients

The attention mechanism ensures that output tokens $y_n$ ave values that are determined not just by the value of the corresponding input token $x_n$, but on *all other* input tokens in the set $\{x_n\}$.

We may express the output of an attention layer $y_n$ as a linear combination of the input tokens:
$$y_n = \sum_{m=1}^N a_{nm} x_m$$
Where the coefficients $a_{nm}$ are called *attention weights*. The scale of the attention weights is then directly related to the significance of the input token on the output. For instance, if $a_3$ is very small, then $x_3$ will contribute very little to the value of $y_n$. Thus, the attention weights are typically constrained as:
$$a_{nm} \ge 0, \ \ \sum_{m=1}^N a_{nm} = 1$$
This ensures that token importance *is relative*; i.e., a token is only more important if another token is made less important.\
At the extremes, all attention weights may be equivalent, indicating equal significance of all features in $x_n$. Likewise, one attention weight may equal $1$, in which case no transformation occurs and $y_n = x_n$.

## 12.1.3 Self-Attention

### Key, Query, Value Primitives

Self-attention is based on notions of keys, queries, and values. These values *determine* the attention weights.\
These are a bit abstract, but I'll firm them up later.

**Value**\
The value is information about the token $x_n$ itself. This information is independent of any other tokens. 

**Key**\
The key is information about the token $x_n$ *in relation to* other tokens. This is analogous to a key in a relational database.

**Query**\
Th query is information about tokens $x_m$ ( $m \ne n$) that relates $x_m$ to $x_n$. Again, this is analogous to a query in relational databases.

At a very basic level, we may allow the key and value of $x_n$ to *both be* $x_n$ itself and the query to be the token $x_m$ itself. Then the attention weights are determined by the similarity of the keys to the queries, normalized to a probability space (via softmax):
$$a_{nm} = \frac{\exp(x_n^\intercal x_m)}{\sum_i \exp(x_n^\intercal x_i)}$$
In matrix notation:
$$Y = \text{Softmax}[XX^\intercal]X$$

This type of self-attention is called ***dot-product self-attention***

## 12.1.4 Network Parameters

Now we need to incorporate learnable parameters into the self-attention mechanism. These parameters will modulate the strength of relationships between the keys $x_n$ and queries $x_m$ in the input $X$, enabling the transformer to emphasize some features over others. To do this, we define matrices of learnable weights for the keys, queries, and values:
$$W^{(k)} \in\reals^{D,D_k}, \ \ W^{(q)} \in\reals^{D,D_q}, \ \ W^{(v)} \in\reals^{D,D_v}$$
Note that we can define the dimensionality of these matrices. Then,
$$Y = \text{Softmax}[XW^{(q)} (W^{(k)})^\intercal X^\intercal] XW^{(v)} = \text{Softmax}[QK^\intercal] V$$

## 12.1.5 Scaled Self-Attention

The dot-product elements of $QK^\intercal$ can become very large depending on the magnitude of the input token vectors $x_n$. Large elements can lead to very large gradients:
- The partial derivative of the softmax for element $x_n$ is $D_nS_n = S_n(1-S_n)$ (following from the quotient rule of derivatives).\
Thus for very large $x_n$ relative to $\{x_{m\ne n}\}$, $S_n \rightarrow 1$ and $(1 - S_n) \rightarrow 0$, causing the gradient to ***vanish***

This vanishing gradient problem is addressed by scaling the dot-products such that none is too large:
$$Y = \text{Softmax}\bigg[\frac{QK^\intercal}{\sqrt{D_k}}\bigg]V$$


## Self-Attention Expounded

We begin with an $N$ length sequence of input tokens $x_n$, each of dimension $D$, denoted by matrix $X$. Then,\
$Q = XW^{(q)}$ is the $N\times D_q$ representation of the input tokens in a $D_q$-dimensional query space.\
$K = XW^{(k)}$ is the $N\times D_k$ representation of the input tokens in a $D_k$-dimensional key space.\
$V=XW^{(v)}$ is the $N\times D_v$ representation of the input tokens in a $D_v$-dimensionl value space.

$QK^\intercal$ is a **square** $D_q \times D_k$ matrix (dimensions must be equal) comprised of pairwise dot-products of each key vector and each query vector. These dot-products are proportional to the cosine similarity between each key-query pair. Thus, tokens $x_n$ and $x_m$ that have key and query representations that are *close together* will have larger weights than those that are further apart.

Moreover, because the key and query parameters are independently learnable, the key-to-query relationship between two tokens may have a *different magnitude* than the query-to-key relationship!\
E.g., if one token encodes the word `'hammer'` $\coloneqq x_h$ and another encodes the word `'tool'` $\coloneqq x_t$ then $q_{t}k_h^\intercal$ is the dot-product of the query representation of `'tool'` and the key representation of `'hammer'`. Likewise, $q_h k_t^\intercal$ is the dot-product of the query representation of `'hammer'` and the key representation of `'tool'`.\
These are **independently learnable**. So, throguh parameter tuning, the magnitude of the relationship between $q_t$ and $k_h$ may be different than the magnitude of the relationship between $q_h$ and $k_t$. This is precisely the behavior we'd hope for, since "hammer" and "tool" can be used in different contexts independently of eachother. (all hammers are tools, not all tools are hammers, etc.)

Thus self-attention can learn ***asymmetric*** relationships between tokens depending on their key and query representations.

Finally, the scaled and softmax transformed $QK^\intercal$ representations are composed with the value representations $V$. These representations preserve some of the original semantic information of the tokens. In a sense, the value representation contains "what" information is passed forwards, while $Q$ and $K$ determine "how much" information to pass forwards. This is seen by examining the backward pass, denoting the softmax term as $A$ (for attention):
$$W_v = W_v - \text{lr} \cdot \frac{\partial L}{\partial W_v} = W_v - \text{lr} \cdot \bigg(\frac{\partial L}{\partial \text{Output}} \cdot A\bigg) \cdot x^\intercal$$
This means that the *training signal* for $V$ is directly shaped by the attention patterns in $A$ despite $V$ itself having no direct influence on the attention patterns.\
In essence, the value matrix $V$ *learns* to create representations that, *when weighted according to the attention patterns* in $A$, produce effective outputs. If certains tokens receive higher attention weights, then the corresponding token-mappings in $W_v$ will be updated more aggressively.

There is a feedback loop here:
- If a token's value representation is useful, models learn to attend to it more
- If attention to a token is consistently high, its value representation becomes more refined


## 12.1.6 Multi-Head Attention

Often, the attention mechanism is partitioned between multiple lower-dimensional subspaces. This enables parallelism in training as well as discriminatory attention, which is a pretty big deal. Define head $h$ as:
$$H_h = \text{Attention}(Q_h, K_h, V_h)$$
Where the output of each head has dimension $N\times D_{vh}$. Typically, we set all heads to have the same dimension $D_{vh}$ denoted $D_v$. Then, the direct sum of the heads' subspaces is given by their concatenation:
$$Y(X)=\text{Concat}[H_1, ... , H_H]W^{(o)}$$
When the heads all have dimensionality $N\times D_v$, the result of the concatenation has dimensionality $N\times HD_v$. This is then transformed by an *output layer*, the mapping $W^{(o)}$ of dimension $HD_v\times N$ to give output $Y$.

When all heads are equal partitions of the feature space $D$, then $D$ is a *direct sum* of the subspaces spanned by each head. For example, partitioning $D=64$ across 2 heads, the first would take the first $32$ dimensions and the second would take the last $32$.

Note that each head may attend to different features in $D$. Allowing the network to conduct more *fine-grained* learning across the feature space.

## 12.1.7 Transformer Layers

Now that we've worked through self-attention, we can define the full transformer layer (this follows the *Attention Is All You Need* formulation):
$$Z = \text{LN}\big[Y(X) + X\big]$$
Where $\text{LN}$ denotes a Layer-Normalization layer, and $Y(X)\text{Concat}[H_1, ... , H_H]W^{(o)}$.\
Sometimes we may see *pre-normalization* instead, wherein:
$$Z = Y(\text{LN}(X)) + X$$
Note that adding $+X$ acts as a residual connection, which allows output values to flow directly back to gradients of earlier layers.

Then, the transformer layer is completed by passing this output through another feed-forward (or "MLP") layer, to yield output $\tilde{X}$:
$$\tilde{X} = \text{LN}\big[\text{MLP}(Z) + Z\big]$$

And thats all there is to it...

## 12.1.9 Positional Encoding

The weight matrices $W^{(q)}$, $W^{(k)}$, and $W^{(v)}$ are shared across all input tokens in $X$. So, changing the order of input tokens (rows) in $X$ changes the order of output representations (rows) in $\tilde X$. The transformer is *equivariant* w.r.t. permutations of the input. This can be good in some situations, but very bad in others. Naturally, this is detrimental for NLP tasks where sequences of input tokens like words matters. For example, the sentence "the food was bad, not good at all" and the sentence "the food was good, not bad at all", would result in identical outputs $\tilde X$ with only the rows corresponding to "good" and "bad" being swapped. Thus, we need to encode positional information into the input such that the transformer can learn that the position of the words "good" and "bad" matters!

Often, this is simply done by creating a second *positional embedding* of the input sequence and simply ***adding*** it to the token embedding: $$\tilde x_n = x_n + r_n$$
Where $r_n$ is the position embedding.

Some notes on this:
- When the dimension of the embedding space is high, $x_n$ and $r_n$ will be approximately orthogonal at random initialization, making their addition a bit cleaner
- The residual connections throughout the transformer ensure that positional embedding information is persisted throughout the network (doesn't get lost in transformations)

Bishop goes on to desribe positional embeddings initialized using sin and cosine waves. However, it is most common (and easiest), to just randomly initialize these embeddings and allow the network to learn them.

# 12.2 Natural Language

## 12.2.1 Word Embedding

Let $K$ be the size of a vocabulary and $D$ be the dimensionality of the embedding space. Then the embedding of the $n^{th}$ word in the vocabulary is given by teh vector $v_n$ defined by:
$$v_n = Ex_n$$
Where $x_n$ is a one-hot vector encoding of the $n^{th}$ word and $E$ is a $K\times D$ matrix representation of the linear map from the vocabulary to the embedding space.

Learning $E$ is a problem of ***Embedding Learning***

### word2vec

`word2vec` learned effective word embeddings through ***self-supervised*** learning. A large corpus of text was collected, from which random samples of $M$ adjacent words were drawn. From this, two different approaches were taken:
1. *Continuous Bag of Words*
    - Predict the *middle word* as the target in each sample, treating the surrounding words as inputs
2. *Skip-Grams*
    - Treat the middle word as the input and predict the *surrounding words* as output targets
    
In both approaches the loss function is the sum of losses across all samples.

A simple two-layer network was trained on this task. The embedding matrix $E$ was then derived as the second layer's weights (in the continuous bag of words approach), or as the first layer's weights (in the skip-grams approach).

## 12.2.2 Tokenization

meh

## 12.2.3 Bag of Words

These models are so named because they completely ignore the ordering of words (or tokens) in the data. At their most basic, they assume that the appearance of a "bag" of words is a joint distribution of the probabilities of each word's occurance. Assuming that these probabilities are IID:
$$p(x_1,...,x_n) = \prod_{n=1}^N p(x_n)$$
An MLE solution is then to simply set the probability of each word $n$, $p(x_n)$, as its frequency of occurances in the data set.

We can augment this slightly to a naive Baye's classifier for classes with different distributions. So, the probability of a bag of words appearing given that they belong to class $C_k$ is:
$$p(x_1,...,x_N | C_k) = \prod_{n=1}^N p(x_n | C_k)$$
And (by Baye's theorem), the posterior class probabilities given a bag of words is then:
$$p(C_k | x_1,...,x_N) \propto p(C_k)\prod_{n=1}^N p(x_n|C_k)$$

## 12.2.4 Autoregressive Models

Autoregression improves upon bag-of-word models by considering word order. We can start by expressing the probability of a sequence of words as a product of each word's conditional probability, conditioned upon all prior words:
$$p(x_1,...,x_N) = \prod_{n=1}^N p(x_n | x_1,...,x_{n-1})$$

This can be approximated (and made massively more efficient) by truncating the conditional probabilities to only include the last $L$ words. E.g., for $L=2$: 
$$p(x_1,...,x_N) = p(x_1)p(x_2|x_1)\prod_{n=3}^N p(x_n | x_{n-1}, x_{n-2})$$

This is a *tri-gram* model, it is part of a general class of *n-gram* models whre *n* is determined by the hyperparameter $L$. 

## 12.2.5 Recurrent Neural Networks

Fundamentally, RNNs are structured as chains of recurrent layers, each of which takes as input a word (token), $x_n$, from the input sequence as well as the hidden-state output, $z_{n-1}$, from the last layer. It then returns an output $y_n$ *as well as* a new hidden-state output $z_n$ to pass as input to the next recurrent layer.

RNNs have several drawbacks, not least of which is that they get quite large for long sequences and suffer from vanishing or exploding gradients because of their depth. A more fundamental problem, though, is that the hidden-state representation $z^*$ for a sequence fundamentally performs a kind of compression. It compresses all of the information in a sequence into one high-dimensional vector of fixed-length. For long sequences, this becomes prohibitive. It is known as ***the bottleneck problem*** - a sequence of arbitrary length must be summarized in a single hidden vector of activations. 

More complex RNN architectures address the bottleneck problem by incorporating new paths that bypass many sub-layers of each recurrent network block, or that otherwise moderate what information is *remembered* from further in the past. These include LSTMs and ***gated recurrent unit*** (GRU) models. But they still have many of the same imperfections and (imo) are messy as hell.

# 12.3 Transformer Language Models

Three broad categories:
1. **Encoder**
    - Take sequences or ensembles or input tokens and return sing-valued outputs. E.g., sentiment analyzer that analyzes sets of text and returns a sentiment category of "happy", "sad", etc.
    - More strictly, defined by singular output (idk that input is necessarily non-singular)
2. **Decoder**
    - Output sets or sequences
3. **Sequence-to-sequence**
    - Both input and output are sequences
    - Often blend encoders and decoders

## 12.3.1 Decoder Transformers

Take input (sequences or otherwise) and output sequences. Uses GPT as an example.\
GPT takes input token sequence $x_1,...,x_N$ and outputs a sequence $\tilde x_1,..., \tilde x_N$ of token logits.
This sequence comprises $\tilde X$ in $N\times D$. We can take a softmax activation over these tokens mapped into a $K$ dimensional sampling space (where $K$ is the vocabulary size) to sample tokens:
$$Y = \text{Softmax}\big(\tilde X W^{(p)}\big)$$

The GPT architecture is autoregressive with the object of predicting the next token in a sequence. Therefore, in training we may use the input sequences shifted by one index to the right as the target labels. I.e., input $x_1,...,x_n$ is scored against targets $x_2, ..., x_{n+1}$.

Additionally, because the model's objective is to predict the next token using prior tokens, the attention mechansim must be restricted to only consider information at prior indices. This is done with ***masking***. Basically, we zero-out the attention weights that correspond to a token attending to any later token in the sequence. This is easily achieved by zeroing-out the *upper triangle* of the attention matrix. Practically, we set the upper-triangular elements to $-\infty$ so that the softmax evaluates to zero for tokens with those attention weights.

Masked attention layers are sometimes called ***causal attention***

## 12.3.2 Sampling Strategies

How can we best select the predicted output token given the softmax probabilities? We could draw from a multinomial sample with probabilities given by the softmax distribution. Or we could simply draw the token with the highest probability each time. This approach would make generation deterministic because the same input sequence would always generate the same output sequence. This is a form of *greedy* search.

Maximizing the probability over the entire generated sequence is often infeasible because it requires computing the joint pdf over all tokens at all indices, which scales exponentialy in complexity, $O(K^N)$.\
Some techniques attempt to implement less greedy searches...

***Beam Search***\
We maintain a set of $B$ hypotheses ($B$ is called the *beam width*), each consisting of a sequence of token values up to step $n$. We then feed all $B$ sequences through the network and find the $B$ most probably token values for each sequence. This creates $B^2$ possible hypotheses for the extended sequence. Then, we prune this set by selecting the $B$ most probable hypotheses according to the *total probability of the extended sequence*.\
So, beam search maintains $B$ alternative sequences and tracks their probabilities before finally selecting the most probable sequence of those considered.

Other approaches include *top-$K$* sampling - i.e. simply sample from the top $K$ greatest probabilities. Or, sampling with ***Temperature***. This approach adds a temperature parameter $T$ to the softmax function:
$$y_i = \frac{\exp(a_i / T)}{\sum_j \exp(a_j/T)}$$
As $T\rightarrow 0$, the probability mass becomes concentrated on the most probable outcome. At $T=1$ the softmax is unmodified. Then, as $T\rightarrow \infty$, the distribution becomes uniform across all outcomes.

## 12.3.3 Encoder Transformers

With encoders, we take sets or sequences and output fixed-length vectors like class labels. A principal example is ***BERT***, bi-directional encoder representations from transformers (Devlin et al., 2018).

BERT prepends each input sequence with a special `<class>` token which is ignored during pretraining. Then, during pre-training, the input sequences are partially masked at random (like in `word2vec`) with special `<mask>` tokens. The masked input tokens are then used as target labels for training. 

These `<mask>` tokens are often diluted somewhat. Delvin et al. (2018) masked 15% of tokens at random. Of this 15%, 10% were replaced with other randomly selected words, and another 10% were left with the true word (though they still had to predict that word). These modifications were made because the model would never see `<mask>` tokens during subsequent fine-tuning applications.

BERT is *bi-directional* because no masking or sequence shifting is done. All tokens may *attend* to all others in the attention layers.

For inference, BERT is *fine-tuned* by adding an output layer specific to the inference task. For text classification, only the first output position is used, the one corresponding to the `<class>` token. If the embedding dimension is $D$ and the number of classes is $K$, then the output layer has dimension $D\times K$. Then, a softmax over the output of this layer.

Alternative variants may use a full MLP instead of a single fine-tuning output layer, or may use a linear-plus-softmax layer to classify all tokens in the sequence simultaneously.